# importando Base de Dados com Fotos dos Atores

In [12]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vishesh1412/celebrity-face-image-dataset")

print("Path to dataset files:", path)
# Lista as pastas de cada celebridade
!ls "$path/Celebrity Faces Dataset"

Using Colab cache for faster access to the 'celebrity-face-image-dataset' dataset.
Path to dataset files: /kaggle/input/celebrity-face-image-dataset
'Angelina Jolie'     'Johnny Depp'	  'Nicole Kidman'	'Tom Hanks'
'Brad Pitt'	     'Kate Winslet'	  'Robert Downey Jr'	'Will Smith'
'Denzel Washington'  'Leonardo DiCaprio'  'Sandra Bullock'
'Hugh Jackman'	     'Megan Fox'	  'Scarlett Johansson'
'Jennifer Lawrence'  'Natalie Portman'	  'Tom Cruise'


In [13]:
!unzip /kaggle/input/celebrity-face-image-dataset


unzip:  cannot find or open /kaggle/input/celebrity-face-image-dataset, /kaggle/input/celebrity-face-image-dataset.zip or /kaggle/input/celebrity-face-image-dataset.ZIP.


## Serão Mantidos 4 atores somente do dataset original


In [14]:
import os
import shutil

# Definir os caminhos

# Caminho da pasta original do Kaggle
source_dir = "/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/"

# Nome da nova pasta de destino que vamos criar
destination_dir = "dados_selecionados"

# Listar os atores que serao mantidos
atores_para_manter = [
    "Brad Pitt",
    "Angelina Jolie",
    "Will Smith",
    "Tom Hanks"
]

# Criar a pasta de destino
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)
    print(f"Pasta '{destination_dir}' criada com sucesso!")

# Copiar as pastas dos atores selecionados
print("\nIniciando a cópia dos diretórios...")
for ator in atores_para_manter:
    # Monta o caminho completo da pasta de origem do ator
    source_ator_path = os.path.join(source_dir, ator)

    # Monta o caminho completo da pasta de destino do ator
    destination_ator_path = os.path.join(destination_dir, ator)

    # Copia a pasta inteira (e seu conteúdo)
    if os.path.exists(source_ator_path):
        print(f"Copiando pasta: {ator}...")
        shutil.copytree(source_ator_path, destination_ator_path)
    else:
        print(f"Aviso: Pasta para '{ator}' não encontrada na origem.")

print("\nCópia concluída!")

# Validar copia
print(f"\nConteúdo da pasta '{destination_dir}':")
!ls -l dados_selecionados/


Iniciando a cópia dos diretórios...
Copiando pasta: Brad Pitt...
Copiando pasta: Angelina Jolie...
Copiando pasta: Will Smith...
Copiando pasta: Tom Hanks...

Cópia concluída!

Conteúdo da pasta 'dados_selecionados':
total 16
drwxr-sr-x 2 root root 4096 Sep 18 17:16 'Angelina Jolie'
drwxr-sr-x 2 root root 4096 Sep 18 17:16 'Brad Pitt'
drwxr-sr-x 2 root root 4096 Sep 18 17:16 'Tom Hanks'
drwxr-sr-x 2 root root 4096 Sep 18 17:16 'Will Smith'


# Agora sera usada a bliblioteca face_recognition para Percorrer "dados_selecionados" e definir a box do rosto em cordenadas em um arquivo.txt


In [15]:
import cv2
import os

# Carregar o classificador Haar Cascade para detecção de face
# OpenCV já vem com vários arquivos cascade pré-treinados
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Definir os caminhos
dataset_path = 'dados_selecionados'
classes = sorted(os.listdir(dataset_path))

print(f"Classes encontradas: {classes}")

# Função para converter coordenadas para o formato YOLO
def to_yolo_format(box, img_width, img_height):
    x, y, w, h = box
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    width = w / img_width
    height = h / img_height
    return f"{x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

# Processar e anotar as imagens
for class_id, class_name in enumerate(classes):
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_path):
        continue

    print(f"\nProcessando a classe: {class_name} (ID: {class_id})")

    for img_name in os.listdir(class_path):
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        image_path = os.path.join(class_path, img_name)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Aviso: Não foi possível carregar a imagem: {image_path}")
            continue

        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img_height, img_width = gray_image.shape

        # Detectar faces na imagem
        # scaleFactor: Parameter specifying how much the image size is reduced at each image scale.
        # minNeighbors: Parameter specifying how many neighbors each candidate rectangle should have to retain it.
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        print(f" - Imagem: {img_name} -> {len(faces)} rosto(s) encontrado(s).")

        if len(faces) > 0:
            annotations = []
            for (x, y, w, h) in faces:
                # A caixa retornada é (x, y, width, height). Converter para o formato YOLO.
                yolo_coords = to_yolo_format((x, y, w, h), img_width, img_height)
                annotations.append(f"{class_id} {yolo_coords}")

            # Salva o arquivo .txt com todas as anotações encontradas
            annotation_path = os.path.splitext(image_path)[0] + '.txt'
            with open(annotation_path, 'w') as f:
                f.write("\n".join(annotations))
        else:
             # Opcional: Criar um arquivo .txt vazio ou com uma mensagem se nenhum rosto for encontrado
             annotation_path = os.path.splitext(image_path)[0] + '.txt'
             with open(annotation_path, 'w') as f:
                 f.write("") # Arquivo vazio se nenhum rosto for detectado


print("\nProcesso de anotação automática concluído usando OpenCV Haar Cascades!")

Classes encontradas: ['Angelina Jolie', 'Brad Pitt', 'Tom Hanks', 'Will Smith']

Processando a classe: Angelina Jolie (ID: 0)
 - Imagem: 064_0de68937.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 053_05c78fa1.jpg -> 2 rosto(s) encontrado(s).
 - Imagem: 001_fe3347c0.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 025_41cee764.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 013_95ecbd39.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 043_b812749f.jpg -> 2 rosto(s) encontrado(s).
 - Imagem: 012_cfcd4007.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 074_0ec79719.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 047_5350c8c0.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 035_2e497561.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 068_2fb110d1.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 078_044866e7.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 026_2828fcaf.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 048_0a32a483.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 079_40a598dc.jpg -> 1 rosto(s) encontrado(s).
 - Imagem: 088_029f

In [16]:
import os
import random

# Caminho para a pasta com os dados selecionados
dataset_path = 'dados_selecionados'

# Lista para guardar apenas os caminhos das imagens válidas (com 1 rosto)
valid_image_paths = []

print("--- Iniciando filtragem e criação dos arquivos de lista de imagens ---")

# Percorre todas as subpastas e arquivos dentro da pasta /content/darknet/data/dados_selecionados
# Usamos os.walk para percorrer as subpastas de cada ator
for root, dirs, files in os.walk(os.path.join("data", dataset_path)):
    for filename in files:
        # Verifica apenas arquivos de imagem
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(root, filename)
            annotation_path = os.path.splitext(image_path)[0] + '.txt'

            # Verifica se a anotação existe e tem apenas uma linha
            if os.path.exists(annotation_path):
                with open(annotation_path, 'r') as f:
                    lines = [line for line in f.read().splitlines() if line.strip()]
                    if len(lines) == 1:
                        # Adiciona o caminho relativo correto para o Darknet
                        valid_image_paths.append(image_path)

print(f"\nEncontradas {len(valid_image_paths)} imagens válidas.")

# Embaralha a lista de imagens
random.shuffle(valid_image_paths)

# Define a proporção da divisão (80% para treino, 20% para validação)
split_ratio = 0.8
split_index = int(len(valid_image_paths) * split_ratio)

# Divide a lista em treino e validação
train_files = valid_image_paths[:split_index]
valid_files = valid_image_paths[split_index:]

# Cria o novo arquivo train.txt
with open('train.txt', 'w') as f:
    for path in train_files:
        f.write(path + '\n')

# Cria o novo arquivo valid.txt
with open('valid.txt', 'w') as f:
    for path in valid_files:
        f.write(path + '\n')

print(f"\nArquivos 'train.txt' e 'valid.txt' recriados com sucesso!")
print(f"Total de imagens para o teste: {len(valid_image_paths)}")
print(f"Imagens de treino: {len(train_files)}")
print(f"Imagens de validação: {len(valid_files)}")

--- Iniciando filtragem e criação dos arquivos de lista de imagens ---

Encontradas 336 imagens válidas.

Arquivos 'train.txt' e 'valid.txt' recriados com sucesso!
Total de imagens para o teste: 336
Imagens de treino: 268
Imagens de validação: 68


In [13]:
# --- CÓDIGO DE CÓPIA ATUALIZADO COM O SEU CAMINHO ---

# 1. Define o seu caminho de destino no Google Drive
# (Não é mais necessário montar o Drive se você já fez isso em uma célula anterior)
DESTINO_DRIVE = "/content/drive/MyDrive/Colab_Notebooks/Projetos_Machine_Learning_DIO/Modulo6_ReconhecimentoFacial/"

print(f"O destino será: {DESTINO_DRIVE}")
# (Opcional, mas recomendado) Garante que a pasta de destino exista
# As aspas são importantes para lidar com nomes de pasta com espaços
!mkdir -p "{DESTINO_DRIVE}"

# 2. Copia a pasta com as imagens e anotações
print("\nCopiando a pasta 'dados_selecionados'...")
!cp -r /content/dados_selecionados "{DESTINO_DRIVE}"

# 3. Copia os arquivos de lista de treino e validação
# (Assumindo que eles estão na pasta /content/darknet/)
print("Copiando train.txt e valid.txt...")
!cp /content/train.txt "{DESTINO_DRIVE}"
!cp /content/valid.txt "{DESTINO_DRIVE}"

# 4. Copia os arquivos de configuração do Darknet
print("Copiando arquivos de configuração (.cfg, .data, .names)...")
!cp /content/darknet/cfg/yolov4-custom.cfg "{DESTINO_DRIVE}"
!cp /content/darknet/data/obj.data "{DESTINO_DRIVE}"
!cp /content/darknet/data/custom/obj.names "{DESTINO_DRIVE}"

print("\nCópia para o seu Google Drive concluída com sucesso! ✅")

# 5. Verifica se os arquivos foram copiados corretamente
print("\nVerificando arquivos na pasta de destino:")
!ls -l "{DESTINO_DRIVE}"

O destino será: /content/drive/MyDrive/Colab_Notebooks/Projetos_Machine_Learning_DIO/Modulo6_ReconhecimentoFacial/

Copiando a pasta 'dados_selecionados'...
cp: cannot stat '/content/dados_selecionados': No such file or directory
Copiando train.txt e valid.txt...
cp: cannot stat '/content/train.txt': No such file or directory
cp: cannot stat '/content/valid.txt': No such file or directory
Copiando arquivos de configuração (.cfg, .data, .names)...
cp: cannot stat '/content/darknet/data/custom/obj.names': No such file or directory

Cópia para o seu Google Drive concluída com sucesso! ✅

Verificando arquivos na pasta de destino:
total 33
drwx------ 7 root root  4096 Sep 18 17:49 dados_selecionados
-rw------- 1 root root   295 Sep 18 19:44 obj.data
-rw------- 1 root root    48 Sep 18 18:08 obj.name
-rw------- 1 root root 12726 Sep 18 18:27 train.txt
-rw------- 1 root root  3231 Sep 18 18:27 valid.txt
-rw------- 1 root root 12248 Sep 18 19:44 yolov4-custom.cfg


In [2]:
# clona o repositório do darknet
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


In [3]:
%cd /content/darknet

# Baixa os pesos convolucionais pré-treinados
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

/content/darknet
--2025-09-18 23:22:58--  https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/75388965/48bfe500-889d-11ea-819e-c4d182fcf0db?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-19T00%3A20%3A30Z&rscd=attachment%3B+filename%3Dyolov4.conv.137&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-18T23%3A20%3A16Z&ske=2025-09-19T00%3A20%3A30Z&sks=b&skv=2018-11-09&sig=SMjy0YmSZOOijDzc2yHfg4vH7gAe8w7eplQk35bNPuU%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1ODIzODA3OCwibmJmIjoxNzU4MjM3Nzc4LCJwYXRoIjoicmVs

In [7]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!make

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:

In [4]:
# 1. Monta o Google Drive (se ainda não estiver montado)
from google.colab import drive
drive.mount('/content/drive')

# 2. Define o caminho da sua pasta de projeto no Drive
PROJETO_PATH = "/content/drive/MyDrive/Colab_Notebooks/Projetos_Machine_Learning_DIO/Modulo6_ReconhecimentoFacial/"

# 3. Copia os arquivos de configuração e dados do seu Drive para as pastas corretas do Darknet

# Copia os arquivos de configuração da rede (.cfg)
!cp "{PROJETO_PATH}yolov4-custom.cfg" /content/darknet/cfg/

# Copia os arquivos de dados (.data e .names)
!cp "{PROJETO_PATH}obj.data" /content/darknet/data/
!cp "{PROJETO_PATH}obj.name" /content/darknet/data/obj.names  # Corrigindo o nome para .names

# Copia os arquivos de lista de imagens (.txt)
!cp "{PROJETO_PATH}train.txt" /content/darknet/data/
!cp "{PROJETO_PATH}valid.txt" /content/darknet/data/

# Copia a pasta de imagens e anotações
!cp -r "{PROJETO_PATH}dados_selecionados" /content/darknet/data/

print("✅ Arquivos de configuração e dados copiados com sucesso para o ambiente do Darknet!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Arquivos de configuração e dados copiados com sucesso para o ambiente do Darknet!


In [11]:
# Recria o arquivo obj.data com os caminhos corretos
# O comando %%writefile cria/sobrescreve um arquivo de texto

%%writefile data/obj.data
classes = 4
train = data/train.txt
valid = data/valid.txt
names = data/obj.names
backup = backup/

print("✅ Arquivo 'obj.data' corrigido com sucesso!")

# Navega para a pasta do darknet, se necessário
%cd /content/darknet

# Mostra o conteúdo do arquivo de configuração
!cat data/obj.data

Overwriting data/obj.data


In [18]:
!mv train.txt data/
!mv valid.txt data/

In [19]:
!./darknet detector train data/obj.data cfg/yolov4-custom.cfg yolov4.conv.137 -map -dont_show

A saída de streaming foi truncada nas últimas 5000 linhas.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.755725), count: 26, class_loss = 2.018271, iou_loss = 4.101268, total_loss = 6.119539 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.783628), count: 18, class_loss = 0.645755, iou_loss = 0.980927, total_loss = 1.626682 
 total_bbox = 500894, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.641507), count: 4, class_loss = 1.226557, iou_loss = 2.074483, total_loss = 3.301041 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.772096), count: 24, class_loss = 2.243290, iou_loss = 3.266317, total_loss = 5.509607 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.751106), count: 18, class_loss = 1.366904, iou_loss = 0.693931, total_loss = 2.060836 
 total_bbox = 500940, rewritten_bbox = 0.

In [5]:
# Lista os primeiros 5 arquivos do seu valid.txt
!head -n 5 /content/darknet/data/valid.txt

dados_selecionados/Will Smith/018_d5d389eb.jpg
dados_selecionados/Brad Pitt/007_74ccfb4a.jpg
dados_selecionados/Tom Hanks/048_043d84fd.jpg
dados_selecionados/Angelina Jolie/095_0be163a1.jpg
dados_selecionados/Brad Pitt/040_9b4bfe5b.jpg


In [6]:
# Certifique-se de que está no diretório correto
%cd /content/darknet

# Executa o detector na imagem de teste
# Certifique-se de que o caminho para os pesos (weights) está correto
!./darknet detector test data/obj.data cfg/yolov4-custom.cfg backup/yolov4-custom_best.weights "/content/darknet/data/dados_selecionados/Brad Pitt/001_c04300ef.jpg" -thresh 0.3

/content/darknet
 GPU isn't used 
 OpenCV version: 4.5.4
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 route  8 2 	                           ->  208 x 208 x 128 
  10 conv     64       1 x 1/ 1    208 x 208 x 128 ->  2

In [26]:
# Define o caminho do seu projeto no Drive
PROJETO_PATH = "/content/drive/MyDrive/Colab_Notebooks/Projetos_Machine_Learning_DIO/Modulo6_ReconhecimentoFacial/"

# Copia seus arquivos de configuração e dados
!cp "{PROJETO_PATH}yolov4-custom.cfg" /content/darknet/cfg/
!cp "{PROJETO_PATH}obj.data" /content/darknet/data/
!cp "{PROJETO_PATH}obj.name" /content/darknet/data/
!cp "{PROJETO_PATH}train.txt" /content/darknet/data/
!cp "{PROJETO_PATH}valid.txt" /content/darknet/data/

# --- PASSO MAIS IMPORTANTE ---
# Copia o ÚLTIMO arquivo de pesos salvo para a pasta principal do darknet
# Este arquivo geralmente está dentro de uma pasta 'backup' no seu Drive
# Verifique o nome correto do arquivo no seu Drive!
!cp /content/drive/MyDrive/yolo-backup/yolov4-tiny-2classes_1000.weights /content/darknet/

In [9]:
# Navega para a pasta do darknet
%cd /content/darknet

# Altera o Makefile para desabilitar a GPU e o CUDNN
!sed -i 's/GPU=1/GPU=0/' Makefile
!sed -i 's/CUDNN=1/CUDNN=0/' Makefile

# Recompila o darknet (este passo é crucial)
!make

/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘buff’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-va

In [15]:
from google.colab import drive
drive.mount('/content/drive')
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet
%cd darknet


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'darknet' already exists and is not an empty directory.
[Errno 20] Not a directory: 'darknet'
/content/darknet


In [10]:
# Importa as bibliotecas necessárias
import cv2
from google.colab.patches import cv2_imshow
# Assuming darknet.py is available in the current directory or sys.path
from darknet import *

# --- 1. Carregar a Rede YOLOv4 Customizada ---
# Certifique-se de que estes caminhos estão corretos
weights_path = "backup/yolov4-custom_best.weights"
config_path = "cfg/yolov4-custom.cfg"
data_path = "data/obj.data"

# Carrega a rede, os nomes das classes e as cores
network, class_names, class_colors = load_network(config_path, data_path, weights_path)
width = network_width(network)
height = network_height(network)

# --- 2. Carregar e Testar a Imagem ---
# SUBSTITUA PELO CAMINHO DA SUA IMAGEM DE VALIDAÇÃO
image_path = "/content/darknet/data/dados_selecionados/Angelina Jolie/001_fe3347c0.jpg"
image = cv2.imread(image_path)

if image is None:
    print(f"Erro: Não foi possível carregar a imagem em {image_path}")
else:
    # Guarda as dimensões originais para recalcular as coordenadas depois
    original_height, original_width, _ = image.shape

    # Prepara a imagem para o formato do Darknet
    darknet_image = make_image(width, height, 3)
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (width, height), interpolation=cv2.INTER_LINEAR)
    # Use copy_image_from_bytes to put the resized image data into darknet_image
    import numpy as np
    copy_image_from_bytes(darknet_image, img_resized.tobytes())


    # Roda a detecção na imagem
    # Define um limite de confiança (ex: 30%)
    confidence_threshold = 0.3
    detections = detect_image(network, class_names, darknet_image, thresh=confidence_threshold)

    # Libera a memória alocada para a imagem Darknet
    free_image(darknet_image)


    # --- 3. Desenhar as Detecções na Imagem ---
    image_with_boxes = np.copy(image) # Cria uma cópia da imagem para desenhar

    for label, confidence, bbox in detections:
        # Converte as coordenadas do YOLO (centro_x, centro_y, largura, altura) para (esquerda, topo, direita, base)
        left, top, right, bottom = bbox2points(bbox)

        # Ajusta as coordenadas para o tamanho da imagem original
        left = int(left * (original_width / width))
        top = int(top * (original_height / height))
        right = int(right * (original_width / width))
        bottom = int(bottom * (original_height / height))


        # Pega a cor da classe (se não encontrar, usa verde)
        color = class_colors.get(label, (0, 255, 0))

        cv2.rectangle(image_with_boxes, (left, top), (right, bottom), color, 2)
        cv2.putText(image_with_boxes, "{} [{:.2f}]".format(label, float(confidence)),
                          (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                          color, 2)

    # --- 4. Exibir o Resultado ---
    print("Exibindo a imagem com as detecções:")
    cv2_imshow(image_with_boxes)

NameError: name 'load_network' is not defined